In [33]:
import numpy as np
import random
from mnist import MNIST
import matplotlib.pyplot as plt

In [34]:
mndata = MNIST('Dataset')
trainX = np.array(mndata.load_training()[0])[:20000]
trainY = np.array(mndata.load_training()[1])[:20000]

#random selection of data for training and validation
valIndices = np.random.choice(len(trainX), 2000)
nonValIndices = [x for x in range(len(trainX)) if x not in valIndices]

valX = trainX[valIndices]
valY = trainY[valIndices]

trainX = trainX[nonValIndices]
trainY = trainY[nonValIndices]

testX = np.array(mndata.load_testing()[0])[:2000]
testY = np.array(mndata.load_testing()[1])[:2000]

In [35]:
def feat(data,i):
    return [1] + data[i].tolist()

def oneHot(clas, noOfClasses):
    feat = np.zeros(noOfClasses)
    feat[clas] = 1;
    return feat

trnX = np.array([feat(trainX,i) for i in range(trainX.shape[0])])/256.0
trnY = np.array([oneHot(trainY[i], 10) for i in range(trainX.shape[0])])

tstX = np.array([feat(testX,i) for i in range(testX.shape[0])])/256.0
tstY = np.array([oneHot(testY[i], 10) for i in range(testX.shape[0])])

valX = np.array([feat(valX,i) for i in range(valX.shape[0])])/256.0
valY = np.array([oneHot(valY[i], 10) for i in range(valX.shape[0])])

In [83]:
def softmax(x):
    x = np.exp(x)
    x = x/x.sum(axis=1)[:, None]
    return x

def error(trnY, y, lamda):
    err = 0    
    for i in range(len(trnY)):
        for k in range(10):
            if(y[i, k] < 0.00001):
                y[i, k] = 0.00001
            elif(y[i, k] > 0.99999):
                y[i, k] = 0.99999
            err += trnY[i, k]*np.log(y[i, k])
    err=-1*err/len(trnY)
    # L2 regularization
    err += lamda * np.linalg.norm(w)**2
    # L1 regularization
#     err += lamda * np.sum(np.abs(w))
    return err

def accuracy(y, y_):
    y_ = np.argmax(y_, axis = 1)
    y = np.argmax(y, axis = 1)
    count = 0
    for i in range(len(y_)):
        if y_[i] == y[i]:
            count += 1
    return count*100.0/len(y_)

In [110]:
err = [[], [], []]
acc = [[], [], []]

lr = 0.0001
w = (np.random.rand(10, 785)-0.5)
w_past = np.zeros((4, 10, 785))
count = 0
past_loss = 0

In [111]:
lamda = [0.01,0.001, 0.0001, 0.00001]

In [113]:
max_acc = 0

In [114]:
#L1 Regularization
err1 = [[], [], []]
acc1 = [[], [], []]
best_lamda = 0
for l in lamda:
    err = [[], [], []]
    acc = [[], [], []]

    lr = 0.0001
    w = (np.random.rand(10, 785)-0.5)
    w_past = np.zeros((4, 10, 785))
    count = 0
    past_loss = 0
    for i in range(300):
        print i
        y = softmax(np.dot(trnX,np.transpose(w)))
        loss = error(trnY, y, l)
        err[0].append(loss)
        err[1].append(error(tstY, softmax(np.dot(tstX,np.transpose(w))),l))
        err[2].append(error(valY, softmax(np.dot(valX,np.transpose(w))),l))
        acc[0].append(accuracy(trnY, y))
        acc[1].append(accuracy(tstY, softmax(np.dot(tstX,np.transpose(w)))))
        acc[2].append(accuracy(valY, softmax(np.dot(valX,np.transpose(w)))))
        if past_loss <= loss and i > 78:
            count += 1
            w_past[0] = w_past[1]
            w_past[1] = w_past[2]
            w_past[2] = w_past[3]
            w_past[3] = w
        elif past_loss > loss and i > 10:
            count = 0
            past_loss = -1
        if count > 3 and i > 10:
            break
        grad = np.dot(np.transpose(trnY-y), trnX)
        # L2
        grad += 2*l*w
        # L1
#         grad += l*np.sign(w)
        w = w + lr*grad
        
    w_f = w_past[0]
    if acc[2][-3] > max_acc:
        best_lamda = l
        max_acc = acc[2][-3]
        w_best = w_f
        err1 = err
        acc1 = acc

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [116]:
w_f = w_past[0]
print "Final testing error = ", err[1][-3]
print "Final training error = ", err[0][-3]
print "Final validation error = ", err[2][-3]

plt.plot([x for x in range(len(err[0]))], err[0], label = "train")
plt.plot([x for x in range(len(err[1]))], err[1], label = "test")
plt.plot([x for x in range(len(err[2]))], err[2], label = "validation")
plt.xlabel("Iterations")
plt.ylabel("Entropy")
plt.title("Using L2 with Lamda = " + str(best_lamda))
plt.legend(loc='upper right', shadow=True)
plt.show()

Final testing error =  0.405393293197
Final training error =  0.2869853516
Final validation error =  0.355717616959


In [117]:

plt.plot([x for x in range(len(acc[0]))], acc[0], label = "train")
plt.plot([x for x in range(len(acc[1]))], acc[1], label = "test")
plt.plot([x for x in range(len(acc[2]))], acc[2], label = "validation")
plt.xlabel("Iterations")
plt.ylabel("Accuracy")
plt.title("Using L2 with Lamda = " + str(best_lamda))
plt.legend(loc='lower right', shadow=True)
plt.show()

In [125]:
plt.imshow(w_best[1][1:].reshape(28, 28))
plt.show()